In [ ]:
import os
import warnings
from dotenv import load_dotenv

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings("ignore")

load_dotenv()

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
import os

pdfs = []
for root, dirs, files in os.walk("rag-dataset"):
    # print(root, dirs, files)
    for file in files:
        if file.endswith(".pdf"):
            pdfs.append(os.path.join(root, file))


In [ ]:
docs = []
for pdf in pdfs:
    loader = PyMuPDFLoader(pdf)
    temp = loader.load()
    docs.extend(temp)

    # print(temp)
    # break

In [ ]:
len(docs)

In [ ]:
# print(docs[0].page_content)
# docs

In [ ]:
from langchain_text_splitters import  RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)

In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model("data")
len(encoding.encode(chunks[0].page_content)), len(encoding.encode(chunks[1].page_content)), len(encoding.encode(docs[1].page_content))

In [ ]:
from langchain_ollama import OllamaEmbeddings

import faiss
from langchain_community.vectorstores import FAISS 
from langchain_community.docstore.in_memory import InMemoryDocstore

In [ ]:
embeddings = OllamaEmbeddings(model='data', base_url='http://localhost:11434')

In [ ]:
vector = embeddings.embed_query("Hello World")

In [ ]:
len(vector)
index = faiss.IndexFlatL2(len(vector))
index.ntotal, index.d

In [ ]:
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [ ]:
vector_store.index.ntotal, vector_store.index.d

In [ ]:
ids = vector_store.add_documents(documents=chunks)

In [ ]:
len(ids), vector_store.index.ntotal

In [ ]:
# ids
# vector_store.index_to_docstore_id


In [ ]:
question = "how to gain muscle mass?"
docs = vector_store.search(query=question, k=5, search_type="similarity")

In [ ]:
docs

In [ ]:
db_name = "data"

vector_store.save_local(db_name)